In [7]:
import pandas as pd

files = ['attack_2012-01-01_2013-05-10_Basile.csv',  # 2013-04-27
         'iran_2012-01-01_2014-04-03_Basile.csv',    # 2013-02-15
         'nigeria_2012-01-01_2013-11-09_Basile.csv', # 2012-09-24
         'pakistan_2012-01-01_2013-12-24_Basile.csv',# 2013-02-13
         'pirates_2012-01-01_2013-12-22_Basile.csv'  # 2013-10-08
        ]

file = files[4]
print(file)
df = pd.read_csv(file, usecols=keep_columns, dtype=defined_types)
print(f'Oldest tweet dates : {df.iloc[df.index[-1]].date}')

pirates_2012-01-01_2013-12-22_Basile.csv
Oldest tweet dates : 2013-10-08


In [ ]:
df.info()

In [10]:
searches = ["abu sayyaf" ,
 "afghanistan" ,
 "agro" ,
 "al-qaeda" ,
 "al-qaeda in the arabian peninsula" ,
 "al-qaeda in the islamic maghreb" ,
 "al-shabaab" ,
 "ammonium nitrate" ,
 "attack" ,
 "biological weapon" ,
 "car bomb" ,
 "chemical weapon" ,
 "conventional weapon" ,
 "dirty bomb" ,
 "eco-terrorism" ,
 "environmental terrorism" ,
 "euskadi ta askatasuna" ,
 "extremism" ,
 "farc" ,
 "fundamentalism" ,
 "hamas" ,
 "hezbollah" ,
 "improvised explosive device" ,
 "iran" ,
 "iraq" ,
 "irish republican army" ,
 "islamist" ,
 "jihad" ,
 "nationalism" ,
 "nigeria" ,
 "nuclear" ,
 "nuclear enrichment" ,
 "pakistan" ,
 "palestine liberation front" ,
 "pirates" ,
 "plo" ,
 "political radicalism" ,
 "recruitment" ,
 "somalia" ,
 "suicide attack" ,
 "suicide bomber" ,
 "taliban" ,
 "tamil tigers" ,
 "tehrik-i-taliban pakistan" ,
 "terror" ,
 "terrorism" ,
 "weapons-grade" ,
 "yemen"]

In [2]:
defined_types = {'id': 'int64',
 'conversation_id': 'int64',
 'date': 'str',
 'time': 'str',
 'user_id': 'int64',
 'username': 'str',
 'name': 'str',
 'tweet': 'str',
 'language': 'category',
 'mentions': 'str', # maybe get rid of
 'urls': 'str',
 'photos': 'str',
 'replies_count': 'int64',
 'retweets_count': 'int64',
 'likes_count': 'int64',
 'hashtags': 'str',
 'link': 'str',
 'retweet': 'bool', # is it a retweet ?
 'video': 'bool',
 'thumbnail': 'str',
 'reply_to': 'str'}

date_cols = ['created_at']

remove_cols = ['near',
               'geo',
               'source',
               'user_rt_id',
               'user_rt',
               'retweet_id',
               'retweet_date',
               'translate',
               'trans_src',
               'trans_dest']
keep_columns = ['id',
 'conversation_id',
 'created_at',
 'date',
 'time',
 'user_id',
 'username',
 'name',
 'tweet',
 'language',
 'mentions',
 'urls',
 'photos',
 'replies_count',
 'retweets_count',
 'likes_count',
 'hashtags',
 'link',
 'retweet',
 'video',
 'thumbnail',
 'reply_to']

In [8]:
# get status

import os
from tqdm import tqdm

dir_files = os.listdir('.')
data_files = [f for f in dir_files if f.endswith('.csv')]
total_count = len(data_files)
count_finished = 0
meta_datasets = {}
keywords = []
mems = []
lengths = []
states = []
last_dates = []
for f in tqdm(data_files):
#     print(f)
    df = pd.read_csv(f, usecols=keep_columns)
    keyword = f.split('_')[0]
    
    mem = df.memory_usage().sum()
    length = len(df.index)
    state = 'finished' if df.iloc[df.index[-1]].date[:-3] == '2012-01' else 'in progress'
    count_finished += 1 if state == 'finished' else 0
    last_date = df.iloc[df.index[-1]].date
    keywords.append(keyword)
    mems.append(mem)
    lengths.append(length)
    states.append(state)
    last_dates.append(last_date)
    
    meta_datasets[keyword] = {'mem': mem, 'length': length, 'state': state, 'last_date': last_date}

    
stats = {'keyword': keywords, 'mem': mems, 'length': lengths, 'state': states, 'last_date': last_dates}
print(f'Finished : {count_finished}/{total_count}')

with open('stats.txt', 'w') as stats_file:
    stats_file.write(repr(meta_datasets))


100%|██████████| 22/22 [10:10<00:00, 27.73s/it]

Finished : 1/22


In [19]:
# sum up all things :
stats_df = pd.DataFrame.from_dict(stats).sort_values('keyword')
all_articles_df = pd.DataFrame(searches, columns=['keyword'])
final_files = [f.split('_')[0] for f in os.listdir('./data') if f.endswith('.gz')]
final_df = pd.DataFrame.from_dict({'keyword': final_files, 'final': ['ok']*len(final_files)})

# join the dataframes
combined_df = stats_df.merge(all_articles_df, how='left', left_on='keyword', right_on='keyword').sort_values('keyword').reset_index(drop=True)
# combined_df.to_csv('progress.csv')
combined_df = combined_df.merge(final_df, how='left', left_on='keyword', right_on='keyword').sort_values('keyword').reset_index(drop=True)
combined_df.sort_values(['keyword', 'last_date'])

,keyword,mem,length,state,last_date,final
0,attack,537224426,3178842,in progress,2013-03-05,NaN
3,attack,117307422,694126,in progress,2013-04-27,NaN
5,attack,1104631448,6536280,in progress,2013-05-10,NaN
4,attack,298804127,1768071,in progress,2013-09-24,NaN
1,attack,1158042546,6852322,in progress,2013-11-06,NaN
2,attack,1130101269,6686989,in progress,2014-04-05,NaN
6,iran,522103320,3089368,in progress,2012-09-25,NaN
8,iran,1307577971,7737147,in progress,2013-02-15,NaN
7,iran,617677383,3654895,in progress,2014-04-03,NaN
11,nigeria,545287078,3226550,finished,2012-01-22,NaN


In [20]:
disk_size = [os.stat(file).st_size for file in data_files]
print(f'Memory (RAM) occupation : {stats_df.mem.sum()/1000000:.1f}MB')
print(f'Disk occupation : {sum(disk_size)/1000000:.1f}MB')

Memory (RAM) occupation : 14367.0MB
Disk occupation : 32627.1MB


In [ ]:
term = "\"abu sayyaf\" OR \"afghanistan\" OR \"agro\" OR \"al-qaeda\" OR \"al-qaeda in the arabian peninsula\" OR \"al-qaeda in the islamic maghreb\" OR \"al-shabaab\" OR \"ammonium nitrate\" OR \"attack\" OR \"biological weapon\" OR \"car bomb\" OR \"chemical weapon\" OR \"conventional weapon\" OR \"dirty bomb\" OR \"eco-terrorism\" OR \"environmental terrorism\" OR \"euskadi ta askatasuna\" OR \"extremism\" OR \"farc\" OR \"fundamentalism\" OR \"hamas\" OR \"hezbollah\" OR \"improvised explosive device\" OR \"iran\" OR \"iraq\" OR \"irish republican army\" OR \"islamist\" OR \"jihad\" OR \"nationalism\" OR \"nigeria\" OR \"nuclear\" OR \"nuclear enrichment\" OR \"pakistan\" OR \"palestine liberation front\" OR \"pirates\" OR \"plo\" OR \"political radicalism\" OR \"recruitment\" OR \"somalia\" OR \"suicide attack\" OR \"suicide bomber\" OR \"taliban\" OR \"tamil tigers\" OR \"tehrik-i-taliban pakistan\" OR \"terror\" OR \"terrorism\" OR \"weapons-grade\" OR \"yemen\""
print('- ' + '\n- '.join(searches))

In [ ]:
%%time
import pandas as pd
# joining dataframes
# , parse_dates=date_cols
df_1 = pd.read_csv('nuclear_2012-01-01_2014-09-01_Basile.csv', usecols=keep_columns, dtype=defined_types)
df_2 = pd.read_csv('nuclear_2012-01-01_2013-11-26_Basile.csv', usecols=keep_columns, dtype=defined_types)
df_3 = pd.read_csv('nuclear_2012-01-01_2013-07-21_Basile.csv', usecols=keep_columns, dtype=defined_types)

print(f'Number of entries in df_1 is : {len(df_1)}')
print(f'Number of entries in df_2 is : {len(df_2)}')
print(f'Number of entries in df_3 is : {len(df_3)}')
print(f'Last date in df_1 is : {df_1.iloc[df_1.index[-1]].date}')
df_1 = df_1.append(df_2)
df_1 = df_1.append(df_3)
df_1 = df_1.reset_index(drop=True)
print(f'Number of entries in df_1 after append is : {len(df_1)}')
print(f'Last date in df_1 after append is : {df_1.iloc[df_1.index[-1]].date}')
print(f'Last date in df_3 is : {df_3.iloc[df_3.index[-1]].date}')

# remove duplicate
print(f'Number of entries before duplicates : {len(df_1)}')
df_1 = df_1.drop_duplicates()
print(f'Number of entries after duplicates : {len(df_1)}')

# save the appended dataframe
df_1.to_csv('data/nuclear_full.gz', index=False, encoding='utf-8')

In [ ]:
%%time
# open dataframe
df_save = pd.read_csv('terrorism_2012-01-01_2014-09-01_Basile.csv', usecols=keep_columns, dtype=defined_types)#, parse_dates=['created_at'])

# remove duplicate
print(f'Number of entries before duplicates : {len(df_save)}')
df_save = df_save.drop_duplicates()
print(f'Number of entries after duplicates : {len(df_save)}')

# save the appended dataframe
df_save.to_csv('data/terrorism_full.gz', index=False, encoding='utf-8')

In [ ]:
df_1.info()
# df_save.info()

In [ ]:
df_save.created_at.sample(10)
# df_1.info(null_counts=True)
# dict(df_1.dtypes)
# df_1[df_1.video==1].sample(10)
# len(a)

In [ ]:
# (df_1.id != df_1.conversation_id).sum()
df_1.columns.to_list()